In [ ]:
using Oceananigans
using Oceananigans.Units
using Oceananigans.OutputReaders: FieldTimeSeries

In [ ]:
Ly = 1000kilometers # north-south extent [m]
Lz = 1kilometers    # depth [m]

grid = RectilinearGrid(size = (48, 8),
                       y = (-Ly/2, Ly/2),
                       z = (-Lz, 0),
                       topology = (Flat, Bounded, Bounded))

In [ ]:
model = HydrostaticFreeSurfaceModel(; grid,
                                    coriolis = BetaPlane(latitude = -45),
                                    buoyancy = BuoyancyTracer(),
                                    tracers = :b,
                                    momentum_advection = WENO(),
                                    tracer_advection = WENO())

In [ ]:
"""
    ramp(y, Δy)

Linear ramp from 0 to 1 between -Δy/2 and +Δy/2.

For example:
```
            y < -Δy/2 => ramp = 0
    -Δy/2 < y < -Δy/2 => ramp = y / Δy
            y >  Δy/2 => ramp = 1
```
"""
ramp(y, Δy) = min(max(0, y/Δy + 1/2), 1)

N² = 1e-5 # [s⁻²] buoyancy frequency / stratification
M² = 1e-7 # [s⁻²] horizontal buoyancy gradient

Δy = 100kilometers # width of the region of the front
Δb = Δy * M²       # buoyancy jump associated with the front
ϵb = 1e-2 * Δb     # noise amplitude

bᵢ(y, z) = N² * z + Δb * ramp(y, Δy) + ϵb * randn()

set!(model, b=bᵢ)

In [ ]:
simulation = Simulation(model, Δt=20minutes, stop_time=30days)

In [ ]:
conjure_time_step_wizard!(simulation, IterationInterval(20), cfl=0.2, max_Δt=20minutes)

In [ ]:
using Printf

wall_clock = Ref(time_ns())

function print_progress(sim)
    u, v, w = model.velocities
    progress = 100 * (time(sim) / sim.stop_time)
    elapsed = (time_ns() - wall_clock[]) / 1e9

    @printf("[%05.2f%%] i: %d, t: %s, wall time: %s, max(u): (%6.3e, %6.3e, %6.3e) m/s, next Δt: %s\n",
            progress, iteration(sim), prettytime(sim), prettytime(elapsed),
            maximum(abs, u), maximum(abs, v), maximum(abs, w), prettytime(sim.Δt))

    wall_clock[] = time_ns()

    return nothing
end

add_callback!(simulation, print_progress, IterationInterval(100))

Diagnostics/Output

In [ ]:
# Output setup
filename = "baroclinic_2D_yz.v2"
u, v, w = model.velocities
ζ = ∂z(v) - ∂y(w)  # Vorticity in y-z plane
b = model.tracers.b

simulation.output_writers[:fields] = JLD2Writer(
    model, (; b, ζ, u, v, w),
    filename=filename * ".jld2",
    schedule=TimeInterval(0.5day),
    overwrite_existing=true
)

In [ ]:
@info "Running the simulation..."

run!(simulation)

@info "Simulation completed in " * prettytime(simulation.run_wall_time)

visualization

In [ ]:
# Visualization
b_ts = FieldTimeSeries(filename * ".jld2", "b")
ζ_ts = FieldTimeSeries(filename * ".jld2", "ζ")

u_ts = FieldTimeSeries(filename * ".jld2", "u")
v_ts = FieldTimeSeries(filename * ".jld2", "v")
w_ts = FieldTimeSeries(filename * ".jld2", "w")

times = b_ts.times

In [ ]:
# Coordinates
y = ynodes(grid, Center())# ./ 1e3  # km
z = znodes(grid, Center())# ./ 1e3  # km

In [ ]:
using CairoMakie
fig = Figure(size=(1800, 1000))

In [ ]:
# Animation setup
n = Observable(1)
b_slice = @lift interior(b_ts[$n], 1, :, :)
ζ_slice = @lift interior(ζ_ts[$n], 1, :, :)

u_slice = @lift interior(u_ts[$n], 1, :, :)
v_slice = @lift interior(v_ts[$n], 1, :, :)
w_slice = @lift interior(w_ts[$n], 1, :, :)

title_text = @lift "Day $(round(times[$n]/day, digits=1))"

In [ ]:
# Animations of Buoyancy and Vorticity



empty!(fig)

# Buoyancy plot
ax_b = Axis(fig[1, 1], title="Buoyancy", xlabel="y [m]", ylabel="z [m]")
hm_b = heatmap!(ax_b, y, z, b_slice, colorrange=(0, Δb), colormap=:thermal)
contour!(ax_b, y, z, b_slice, levels=10, color=:black, linewidth=0.5)
Colorbar(fig[1, 2], hm_b, label="Buoyancy [m s⁻²]")

# Vorticity plot
ax_ζ = Axis(fig[1, 3], title="Vorticity", xlabel="y [m]")
hm_ζ = heatmap!(ax_ζ, y, z, ζ_slice, colorrange=(-3e-3, 3e-3), colormap=:balance)
contour!(ax_ζ, y, z, b_slice, levels=10, color=:black, linewidth=0.5)  # Overlay buoyancy contours
Colorbar(fig[1, 4], hm_ζ, label="Vorticity [s⁻¹]")

Label(fig[0, :], title_text, fontsize=24)

# Create animation
frames = 1:length(times)
record(fig, filename * "_animation.mp4", frames, framerate=8) do i
    n[] = i
end

fig  # Display final frame

In [ ]:
#animations of of V and W commented out

#=

# y velocity (v)

empty!(fig)

ax_v = Axis(fig[1, 1], title="V", xlabel="y [m]", ylabel="z [m]")
hm = heatmap!(ax_v, y, z, v_slice; colorrange=(-1e-1, 1e-1), colormap=:balance)
Colorbar(fig[1, 2], hm_b, label="V [m s⁻1]")
contour!(ax_v, y, z, b_slice, levels=10, color=:black, linewidth=0.5)

# z velocity (w)
ax_w = Axis(fig[1, 3], title="W", xlabel="y [m]")
hm_w = heatmap!(ax_w, y, z, w_slice, colorrange=(-2e-2, 2e-2), colormap=:balance)
contour!(ax_w, y, z, b_slice, levels=10, color=:black, linewidth=0.5)  # Overlay buoyancy contours
Colorbar(fig[1, 4], hm_w, label="W [ms⁻¹]")

Label(fig[0, :], title_text, fontsize=24)

# Create animation
frames = 1:length(times)
record(fig, filename * "_animation_velocities.mp4", frames, framerate=8) do i
    n[] = i
end

fig  # Display final frame 

=#

In [ ]:
#animation of V only

empty!(fig)

ax_v = Axis(fig[1, 1], title="V", xlabel="y [m]", ylabel="z [m]")
hm = heatmap!(ax_v, y, z, v_slice; colorrange=(-1e-1, 1e-1), colormap=:balance)
Colorbar(fig[1, 2], hm_b, label="V [m s⁻1]")
contour!(ax_v, y, z, b_slice, levels=10, color=:black, linewidth=0.5)


# Create animation
frames = 1:length(times)
record(fig, filename * "_animation_velocities.mp4", frames, framerate=8) do i
    n[] = i
end

fig  # Display final frame

In [ ]:
#animation of U only

empty!(fig)

ax_u = Axis(fig[1, 1], title="U", xlabel="y [m]", ylabel="z [m]")
hm = heatmap!(ax_u, y, z, u_slice; colorrange=(-1e-1, 1e-1), colormap=:balance)
Colorbar(fig[1, 2], hm_b, label="V [m s⁻1]")
contour!(ax_u, y, z, b_slice, levels=10, color=:black, linewidth=0.5)


# Create animation
frames = 1:length(times)
record(fig, filename * "_animation_velocities.mp4", frames, framerate=8) do i
    n[] = i
end

fig  # Display final frame